### Find all ETFs from Polygon

In [1]:
import requests
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# Replace with your actual Polygon API key
API_KEY = "FWjYhAphGxZUquLTQxQRF0QpSzaLtOJS"

# Initial URL
url = "https://api.polygon.io/v3/reference/tickers"
params = {
    "type": "ETF",
    "active": "true",
    "limit": 1000,
    "sort": "ticker",
    "apiKey": API_KEY
}

all_etfs = []

while True:
    response = requests.get(url, params=params)
    data = response.json()

    # Check for errors
    if response.status_code != 200 or "results" not in data:
        print("Error:", data)
        break

    all_etfs.extend(data["results"])

    # Get next_url and manually re-attach API key
    next_url = data.get("next_url")
    if not next_url:
        break

    # Rebuild next_url with API key
    parsed_url = urlparse(next_url)
    query = parse_qs(parsed_url.query)
    query["apiKey"] = API_KEY
    new_query_string = urlencode(query, doseq=True)
    url = urlunparse(parsed_url._replace(query=new_query_string))

    # Clear params for next iteration
    params = {}

# Save results to CSV
etf_df = pd.DataFrame(all_etfs)
etf_df.to_csv("polygon_etf_list.csv", index=False)
print(f"Saved {len(etf_df)} ETFs to polygon_etf_list.csv")

Saved 4840 ETFs to polygon_etf_list.csv


### Import Price Data from Yahoo Finance

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os

monthly_file_path = "etf_monthly_metrics.csv" 
sharpe_file_path = "etf_sharpe_summary.csv" 

if os.path.exists(monthly_file_path):
    os.remove(monthly_file_path)
    print(f"File '{monthly_file_path}' deleted successfully.")
else:
    print(f"File '{monthly_file_path}' does not exist.")
    
if os.path.exists(sharpe_file_path):
    os.remove(sharpe_file_path)
    print(f"File '{sharpe_file_path}' deleted successfully.")
else:
    print(f"File '{sharpe_file_path}' does not exist.")

current_year = datetime.now().year
current_month = datetime.now().month
current_day = datetime.now().day

today = str(current_year)+'-'+str(current_month)+'-'+str(current_day)
five_years_ago = str(current_year-5)+'-'+str(current_month)+'-'+str(current_day)

# === CONFIGURATION ===
BATCH_SIZE = 400
RF_ANNUAL = 0.03
RF_MONTHLY = RF_ANNUAL / 12
START_DATE = five_years_ago
END_DATE = today
TICKER_FILE = "polygon_etf_list.csv"
MONTHLY_OUTPUT = "etf_monthly_metrics.csv"
SHARPE_OUTPUT = "etf_sharpe_summary.csv"

# === LOAD TICKERS ===
etf_list = pd.read_csv(TICKER_FILE)
tickers = etf_list['ticker'].dropna().unique().tolist()

# === MAIN LOOP ===
monthly_header_written = False
sharpe_header_written = False

for i in range(0, len(tickers), BATCH_SIZE):
    batch = tickers[i:i+BATCH_SIZE]
    print(f"\n📦 Batch {i//BATCH_SIZE + 1} — Downloading {len(batch)} ETFs...")

    try:
        data = yf.download(
            tickers=batch,
            start=START_DATE,
            end=END_DATE,
            interval="1mo",
            auto_adjust=True,
            group_by='ticker',
            progress=False,
            threads=True
        )
    except Exception as e:
        print(f"❌ Failed to download batch: {e}")
        continue

    monthly_frames = []
    sharpe_records = []

    for ticker in batch:
        try:
            df = data[ticker][['Close']].rename(columns={'Close': 'Adj Close'})
            df['Ticker'] = ticker
            df['Return'] = df['Adj Close'].pct_change()
            df['Volatility'] = df['Return'].rolling(12).std() * np.sqrt(12)
            df['Sharpe Ratio'] = (df['Return'].rolling(12).mean() - RF_MONTHLY) / df['Return'].rolling(12).std()
            df = df.reset_index()

            # Append to monthly data
            monthly_frames.append(df)

            # Compute full-period Sharpe (excluding first 12 rows)
            df_clean = df.iloc[12:].copy()
            if len(df_clean) >= 12:
                excess_ret = df_clean['Return'] - RF_MONTHLY
                avg_ret = excess_ret.mean()
                vol = df_clean['Return'].std()
                sharpe = (avg_ret * 12) / (vol * np.sqrt(12))
                sharpe_records.append({"Ticker": ticker, "5Y Sharpe Ratio": sharpe})
        except Exception as e:
            print(f"⚠️ Skipping {ticker}: {e}")

    # === SAVE MONTHLY DATA ===
    if monthly_frames:
        monthly_combined = pd.concat(monthly_frames, ignore_index=True)
        monthly_combined.to_csv(MONTHLY_OUTPUT, mode='a', header=not monthly_header_written, index=False)
        monthly_header_written = True
        print(f"✅ Saved {len(monthly_combined)} monthly rows.")

    # === SAVE 5Y SHARPE SUMMARY ===
    if sharpe_records:
        sharpe_df = pd.DataFrame(sharpe_records)
        sharpe_df.to_csv(SHARPE_OUTPUT, mode='a', header=not sharpe_header_written, index=False)
        sharpe_header_written = True
        print(f"✅ Saved {len(sharpe_df)} 5Y Sharpe scores.")

    time.sleep(1)  # avoid rate-limiting

File 'etf_monthly_metrics.csv' deleted successfully.
File 'etf_sharpe_summary.csv' deleted successfully.

📦 Batch 1 — Downloading 400 ETFs...



10 Failed downloads:
['BGLIF', 'AMISF', 'AMNRF', 'AXNQF', 'AGVTF', 'AMUTF', 'AUMDF', 'BMOZF', 'ANDXF', 'BGEFF']: YFTzMissingError('possibly delisted; no timezone found')
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: Fu

✅ Saved 24000 monthly rows.
✅ Saved 400 5Y Sharpe scores.

📦 Batch 2 — Downloading 400 ETFs...



5 Failed downloads:
['CFYFF', 'CFATF', 'CSWUF']: YFTzMissingError('possibly delisted; no timezone found')
['DBXKF', 'BXMNF']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2020-8-1 -> 2025-8-1)')
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_

✅ Saved 24000 monthly rows.
✅ Saved 400 5Y Sharpe scores.

📦 Batch 3 — Downloading 400 ETFs...


...

/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to ca

/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to ca

✅ Saved 24000 monthly rows.
✅ Saved 400 5Y Sharpe scores.

📦 Batch 13 — Downloading 40 ETFs...


/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()
/var/folders/90/wngdmn2n3cbbb6969r3ry_6m0000gp/T/ipykernel_19298/1938076694.py:74: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Return'] = df['Adj Close'].pct_change()


✅ Saved 2400 monthly rows.
✅ Saved 40 5Y Sharpe scores.


### Remove Data that has NAs

In [2]:
monthly_df = pd.read_csv("etf_monthly_metrics.csv")

monthly_df = monthly_df.sort_values(['Ticker', 'Date'])

# Define a function to keep only tickers where all values after year 1 are valid
def has_complete_data_after_year1(group):
    if len(group) <= 12:
        return False  # not enough data to evaluate
    after_year1 = group.iloc[12:]  # drop first 12 months
    return after_year1[['Return', 'Volatility', 'Sharpe Ratio']].notnull().all().all()

# Apply the filter
filtered_df = monthly_df.groupby('Ticker').filter(has_complete_data_after_year1)

filtered_df.to_csv("etf_monthly_metrics_filtered.csv", index=False)

### Remove Same ETFs from Sharpe File

In [3]:
sharpe_df = pd.read_csv("etf_sharpe_summary.csv")

filt = pd.DataFrame(filtered_df["Ticker"].unique()).rename(columns={0: 'Ticker'})

sharpe_df_filt = sharpe_df.merge(filt,on="Ticker")

### Classify Remaing ETFs into Asset Classes

In [20]:
# Remaining ETFs
remaining_ETFs = sharpe_df_filt["Ticker"].unique()

# Initialize classification list
etf_classifications = []


def classify_etf(name, category):
    text = (str(name) + " " + str(category)).lower()
    if "bond" in text or "treasury" in text:
        return "Fixed Income"
    elif (
        "real estate" in text
        or "reit" in text
        or "gold" in text
        or "commodity" in text
        or "crypto" in text
    ) and "goldman" not in text:
        return "Alternatives"
    elif (
        "short-term" in text
        or "t-bill" in text
        or "cash" in text
        or "ultra short" in text
    ):
        return "Liquidity"
    elif (
        "allocation" in text
        or "multi-asset" in text
        or "balanced" in text
        or "target" in text
    ):
        return "Blended / Multi-Asset"
    else:
        return "Equities"


# Loop through tickers and classify
for ticker in remaining_ETFs:
    print("classifying: "+ticker)
    try:
        info = yf.Ticker(ticker).info
        name = info.get('shortName', '')
        category = info.get('category', '')
        classification = classify_etf(name, category)
        etf_classifications.append({'Ticker': ticker, 'Name': name, 'Category': category, 'Class': classification})
    except Exception as e:
        etf_classifications.append({'Ticker': ticker, 'Name': '', 'Category': '', 'Class': 'Unclassified'})

df_classified = pd.DataFrame(etf_classifications)

classifying: AAAU
classifying: AADR
classifying: AAVM
classifying: AAXJ
classifying: ABEQ
classifying: ABFL
classifying: ABLG
classifying: ACES
classifying: ACGR
classifying: ACIO
classifying: ACLC
classifying: ACSI
classifying: ACWI
classifying: ACWV
classifying: ACWX
classifying: ADFI
classifying: ADME
classifying: AESR
classifying: AFIF
classifying: AFK
classifying: AFLG
classifying: AFMC
classifying: AFSM
classifying: AGG
classifying: AGGY
classifying: AGNG
classifying: AGZ
classifying: AGZD
classifying: AIA
classifying: AIEQ
classifying: AIQ
classifying: AIRR
classifying: AIVC
classifying: AIVI
classifying: AIVL
classifying: ALTL
classifying: ALTY
classifying: AMLP
classifying: AMOM
classifying: AMZA
classifying: ANGL
classifying: AOA
classifying: AOK
classifying: AOM
classifying: AOR
classifying: ARB
classifying: ARCM
classifying: ARGT
classifying: ARKF
classifying: ARKG
classifying: ARKK
classifying: ARKQ
classifying: ARKW
classifying: ARTY
classifying: ASEA
classifying: ASET
cl

classifying: FDEM
classifying: FDEV
classifying: FDG
classifying: FDHY
classifying: FDIS
classifying: FDL
classifying: FDLO
classifying: FDM
classifying: FDMO
classifying: FDN
classifying: FDNI
classifying: FDRR
classifying: FDT
classifying: FDTS
classifying: FDVV
classifying: FEM
classifying: FEMB
classifying: FEMS
classifying: FENY
classifying: FEP
classifying: FEUZ
classifying: FEX
classifying: FEZ
classifying: FFEB
classifying: FFIU
classifying: FFLC
classifying: FFTY
classifying: FGD
classifying: FGM
classifying: FHLC
classifying: FIBR
classifying: FID
classifying: FIDI
classifying: FIDU
classifying: FILL
classifying: FINX
classifying: FISR
classifying: FITE
classifying: FIVA
classifying: FIW
classifying: FIXD
classifying: FJP
classifying: FJUL
classifying: FJUN
classifying: FKU
classifying: FLAU
classifying: FLAX
classifying: FLBL
classifying: FLBR
classifying: FLCA
classifying: FLCB
classifying: FLCH
classifying: FLCO
classifying: FLDR
classifying: FLEE
classifying: FLEU
classif

classifying: ISSZF
classifying: ISTB
classifying: ISTXF
classifying: ISUDF
classifying: ISUUF
classifying: ISVQF
classifying: ISVUF
classifying: ISVVF
classifying: ISZZF
classifying: ITA
classifying: ITB
classifying: ITEQ
classifying: ITM
classifying: ITOT
classifying: ITSXF
classifying: ITTSF
classifying: IUAGF
classifying: IUNSF
classifying: IUS
classifying: IUSB
classifying: IUSG
classifying: IUSPF
classifying: IUSV
classifying: IUTCF
classifying: IUTSF
classifying: IVAL
classifying: IVATF
classifying: IVBUF
classifying: IVE
classifying: IVLU
classifying: IVOG
classifying: IVOL
classifying: IVOO
classifying: IVOV
classifying: IVSPF
classifying: IVSRF
classifying: IVUSF
classifying: IVV
classifying: IVVPF
classifying: IVVVF
classifying: IVW
classifying: IWB
classifying: IWC
classifying: IWD
classifying: IWF
classifying: IWL
classifying: IWM
classifying: IWN
classifying: IWO
classifying: IWP
classifying: IWR
classifying: IWS
classifying: IWV
classifying: IWX
classifying: IWY
classifyi

classifying: RNEM
classifying: RNRG
classifying: ROAM
classifying: ROBO
classifying: ROBT
classifying: RODM
classifying: ROKT
classifying: ROM
classifying: ROMO
classifying: ROSC
classifying: ROUS
classifying: RPAR
classifying: RPG
classifying: RPV
classifying: RSP
classifying: RSPC
classifying: RSPD
classifying: RSPF
classifying: RSPG
classifying: RSPH
classifying: RSPM
classifying: RSPN
classifying: RSPR
classifying: RSPS
classifying: RSPT
classifying: RSPU
classifying: RTH
classifying: RVNU
classifying: RWJ
classifying: RWK
classifying: RWL
classifying: RWM
classifying: RWO
classifying: RWR
classifying: RWX
classifying: RXD
classifying: RXI
classifying: RXL
classifying: RYLD
classifying: RZG
classifying: RZV
classifying: SAA
classifying: SBB
classifying: SBIO
classifying: SCC
classifying: SCHA
classifying: SCHB
classifying: SCHC
classifying: SCHD
classifying: SCHE
classifying: SCHF
classifying: SCHG
classifying: SCHH
classifying: SCHI
classifying: SCHJ
classifying: SCHK
classifying:

classifying: WOOD
classifying: WSDMF
classifying: WTIPF
classifying: WTMF
classifying: WTPI
classifying: WTRE
classifying: WTV
classifying: WUGI
classifying: WWJD
classifying: XAR
classifying: XBI
classifying: XCEM
classifying: XES
classifying: XHB
classifying: XHE
classifying: XHS
classifying: XITK
classifying: XLB
classifying: XLC
classifying: XLE
classifying: XLF
classifying: XLG
classifying: XLI
classifying: XLK
classifying: XLP
classifying: XLRE
classifying: XLSR
classifying: XLU
classifying: XLV
classifying: XLY
classifying: XME
classifying: XMHQ
classifying: XMLV
classifying: XMMO
classifying: XMPT
classifying: XMTIF
classifying: XMVM
classifying: XNTK
classifying: XOP
classifying: XOVR
classifying: XPH
classifying: XPP
classifying: XRLV
classifying: XRT
classifying: XSD
classifying: XSHD
classifying: XSHQ
classifying: XSLV
classifying: XSMO
classifying: XSOE
classifying: XSVM
classifying: XSW
classifying: XT
classifying: XTL
classifying: XTN
classifying: XTTRF
classifying: XYLD

In [42]:
df_classified.merge(sharpe_df_filt,on="Ticker").to_csv("etf_sharpe_summary_filtered.csv",index=False)